In [6]:
import subprocess
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

Merge Meteo & Marine DataFrames

In [2]:
marine_data = pd.read_csv('df_marine_final.csv')
meteo_data = pd.read_csv('df_meteo_final.csv')

# Convertir la colonne "Datetime" en type datetime si ce n'est pas déjà fait
marine_data['Datetime'] = pd.to_datetime(marine_data['Datetime'])
meteo_data['Datetime'] = pd.to_datetime(meteo_data['Datetime'])

# Fusionner les deux DataFrames en utilisant la colonne "Datetime"
final_df = pd.merge(marine_data, meteo_data, on='Datetime', how='inner')
final_df.head()


,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,Datetime,wind speed (ms),wind dir (°),wind_gusts 1h (ms),T° (C°),T°_max 24h (C°),T° min 24h (°),sea level pressure (hPa),Precip 1h (mm)
0,100.0,2.0,4.0,1.8,15.0,7.3,185.0,1017.0,22.5,24.5,2024-05-21 13:20:00+00:00,1.7,130.8,5.5,22.3,24.1,22.1,1016.6,0.24
1,100.0,2.0,3.0,1.6,8.0,7.0,105.0,1017.0,22.3,24.5,2024-05-21 12:50:00+00:00,1.5,133.0,5.7,22.3,24.1,22.1,1016.7,0.22
2,90.0,2.0,3.0,1.6,8.0,7.0,111.0,1017.2,22.3,24.4,2024-05-21 12:20:00+00:00,2.0,136.0,5.7,22.3,24.1,22.1,1017.0,0.22
3,120.0,2.0,4.0,1.6,15.0,7.4,185.0,1017.4,22.4,24.5,2024-05-21 11:50:00+00:00,2.0,138.3,5.7,22.4,24.1,22.1,1017.4,0.22
4,130.0,4.0,5.0,1.6,15.0,7.4,196.0,1017.7,22.7,24.4,2024-05-21 11:20:00+00:00,4.0,139.7,5.7,22.7,24.1,22.1,1017.8,0.22


Drop Datetime + Mean Pressure : 'Pressure' <--> 'sea level pressure (hPa)'

In [3]:
final_df = final_df.drop(columns=['Datetime'])
# Créer la colonne 'PRESSURE' contenant la moyenne entre 'sea level pressure (hPa)' et 'Pressure'
final_df['Pressure'] = (final_df['sea level pressure (hPa)'] + final_df['Pressure']) / 2
final_df = final_df.drop(columns=['sea level pressure (hPa)'])
final_df.head()

,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,wind speed (ms),wind dir (°),wind_gusts 1h (ms),T° (C°),T°_max 24h (C°),T° min 24h (°),Precip 1h (mm)
0,100.0,2.0,4.0,1.8,15.0,7.3,185.0,1016.80,22.5,24.5,1.7,130.8,5.5,22.3,24.1,22.1,0.24
1,100.0,2.0,3.0,1.6,8.0,7.0,105.0,1016.85,22.3,24.5,1.5,133.0,5.7,22.3,24.1,22.1,0.22
2,90.0,2.0,3.0,1.6,8.0,7.0,111.0,1017.10,22.3,24.4,2.0,136.0,5.7,22.3,24.1,22.1,0.22
3,120.0,2.0,4.0,1.6,15.0,7.4,185.0,1017.40,22.4,24.5,2.0,138.3,5.7,22.4,24.1,22.1,0.22
4,130.0,4.0,5.0,1.6,15.0,7.4,196.0,1017.75,22.7,24.4,4.0,139.7,5.7,22.7,24.1,22.1,0.22


In [4]:
final_df.head()

,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Dominant Wave Period (s),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,wind speed (ms),wind dir (°),wind_gusts 1h (ms),T° (C°),T°_max 24h (C°),T° min 24h (°),Precip 1h (mm)
0,100.0,2.0,4.0,1.8,15.0,7.3,185.0,1016.80,22.5,24.5,1.7,130.8,5.5,22.3,24.1,22.1,0.24
1,100.0,2.0,3.0,1.6,8.0,7.0,105.0,1016.85,22.3,24.5,1.5,133.0,5.7,22.3,24.1,22.1,0.22
2,90.0,2.0,3.0,1.6,8.0,7.0,111.0,1017.10,22.3,24.4,2.0,136.0,5.7,22.3,24.1,22.1,0.22
3,120.0,2.0,4.0,1.6,15.0,7.4,185.0,1017.40,22.4,24.5,2.0,138.3,5.7,22.4,24.1,22.1,0.22
4,130.0,4.0,5.0,1.6,15.0,7.4,196.0,1017.75,22.7,24.4,4.0,139.7,5.7,22.7,24.1,22.1,0.22


Export DataFrame to BigQuery

In [5]:
# Définir les identifiants de projet et de dataset
dataset_id = 'rare-bloom-419220.Machine_Learning'
project_id = 'rare-bloom-419220'
table__bq_meteo = "Buoy_Meteo"
table__bq_marine = "Buoy_Current"

# Get Credentials
big_query_credentials = service_account.Credentials.from_service_account_file(
'Big_Query_Connect_Key.json')

client = bigquery.Client(credentials= big_query_credentials,project=project_id)




def upload_dataframe_to_bigquery(client, table_ref, dataframe):
    # Supprimer la table si elle existe déjà
    client.delete_table(table_ref, not_found_ok=True)

    # Configuration du chargement de la table
    load_job_config = bigquery.LoadJobConfig()
    load_job_config.autodetect = True  # Auto-détection du schéma
    load_job_config.skip_leading_rows = 1  # Ignorer la première ligne (entête)

    # Charger la DataFrame dans BigQuery
    job = client.load_table_from_dataframe(dataframe, table_ref, job_config=load_job_config)
    
    # Attendre la fin du chargement
    job.result()

    print("Data uploaded to BigQuery successfully.")



# Définir le nom de la table dans BigQuery

# Créer une référence à la table BigQuery
table_ref = bigquery.TableReference(dataset_id, table_name)

# Charger la DataFrame dans BigQuery
upload_dataframe_to_bigquery(client, table_ref, df_marine)


NameError: name 'table_name' is not defined